In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"

  # IPython 7.14 deprecated set_matplotlib_formats from IPython
  try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
  except ImportError:
    # Fall back to deprecated location for older IPython versions
    from IPython.display import set_matplotlib_formats
    
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL2pvaG5jaGVuL0RvY3VtZW50cy9Tb2x1dGlvbnMvY291cnNlLXdlYnNpdGUvc2xpZGVzLzAy'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/importlib/_bootstrap.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/importlib/_bootstrap_external.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/zipimport.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/codecs.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/encodings/aliases.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/encodings/__init__.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/encodings/utf_8.py": 1755169931.0, "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.

In [2]:
import pandas as pd

# Creating a Series
ser = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

# Accessing elements
print("a:", ser['a'])

a: -5


In [3]:
# Creating a DataFrame
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9]
}
frame = pd.DataFrame(data)

# Selecting columns
print(frame['state'])

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
Name: state, dtype: object


In [4]:
# Index objects
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index

print(index)

Index(['a', 'b', 'c'], dtype='object')


In [5]:
# Loading data from CSV
df = pd.read_csv('data/rent_avg.csv')

df

,RegionID,SizeRank,RegionName,RegionType,StateName,1/31/15,2/28/15,3/31/15,4/30/15,5/31/15,...,2/28/23,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23,8/31/23,9/30/23,10/31/23,11/30/23
0,102001,0,United States,country,NaN,1266.059583,1272.748070,1281.390109,1291.808026,1301.544232,...,2072.346516,2084.944938,2100.570959,2113.158286,2123.032953,2132.040398,2141.677753,2147.835795,2148.939168,2147.563754
1,394913,1,"New York, NY",msa,NY,2233.133615,2255.035180,2272.077073,2291.645864,2297.479956,...,3336.961840,3402.179034,3470.949450,3492.930681,3502.786897,3503.650740,3509.480232,3493.537322,3460.942251,3445.696877
2,753899,2,"Los Angeles, CA",msa,CA,2571.296547,2586.050819,2604.348963,2616.104497,2637.303435,...,4073.810818,4100.234089,4134.999768,4148.832674,4182.522537,4202.376930,4230.003153,4224.038689,4213.465460,4209.636932
3,394463,3,"Chicago, IL",msa,IL,1504.096116,1510.879827,1522.416987,1534.343702,1547.516576,...,2112.639599,2123.617285,2145.692631,2163.843271,2183.541315,2196.506806,2213.427631,2225.237153,2225.798038,2221.960828
4,394514,4,"Dallas, TX",msa,TX,1363.557414,1371.136919,1381.114797,1394.643185,1408.025840,...,2222.442779,2229.474165,2239.462332,2253.004851,2267.408242,2280.056798,2285.063932,2284.569053,2274.785931,2277.403767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,753871,811,"Breckenridge, CO",msa,CO,NaN,NaN,NaN,NaN,NaN,...,3848.649613,4327.112762,4414.482838,4438.619592,4535.664564,4599.237795,4558.170264,4667.043338,4671.081209,4472.222222
468,394751,821,"Kirksville, MO",msa,MO,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.733430,981.701456,951.458333
469,753923,849,"The Dalles, OR",msa,OR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1757.739178,1788.943167,1885.904065,1838.888889
470,394584,863,"Fallon, NV",msa,NV,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1295.000000


In [6]:
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,1/31/15,2/28/15,3/31/15,4/30/15,5/31/15,...,2/28/23,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23,8/31/23,9/30/23,10/31/23,11/30/23
0,102001,0,United States,country,NaN,1266.059583,1272.748070,1281.390109,1291.808026,1301.544232,...,2072.346516,2084.944938,2100.570959,2113.158286,2123.032953,2132.040398,2141.677753,2147.835795,2148.939168,2147.563754
1,394913,1,"New York, NY",msa,NY,2233.133615,2255.035180,2272.077073,2291.645864,2297.479956,...,3336.961840,3402.179034,3470.949450,3492.930681,3502.786897,3503.650740,3509.480232,3493.537322,3460.942251,3445.696877
2,753899,2,"Los Angeles, CA",msa,CA,2571.296547,2586.050819,2604.348963,2616.104497,2637.303435,...,4073.810818,4100.234089,4134.999768,4148.832674,4182.522537,4202.376930,4230.003153,4224.038689,4213.465460,4209.636932
3,394463,3,"Chicago, IL",msa,IL,1504.096116,1510.879827,1522.416987,1534.343702,1547.516576,...,2112.639599,2123.617285,2145.692631,2163.843271,2183.541315,2196.506806,2213.427631,2225.237153,2225.798038,2221.960828
4,394514,4,"Dallas, TX",msa,TX,1363.557414,1371.136919,1381.114797,1394.643185,1408.025840,...,2222.442779,2229.474165,2239.462332,2253.004851,2267.408242,2280.056798,2285.063932,2284.569053,2274.785931,2277.403767


In [7]:
df.tail()

,RegionID,SizeRank,RegionName,RegionType,StateName,1/31/15,2/28/15,3/31/15,4/30/15,5/31/15,...,2/28/23,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23,8/31/23,9/30/23,10/31/23,11/30/23
467,753871,811,"Breckenridge, CO",msa,CO,NaN,NaN,NaN,NaN,NaN,...,3848.649613,4327.112762,4414.482838,4438.619592,4535.664564,4599.237795,4558.170264,4667.043338,4671.081209,4472.222222
468,394751,821,"Kirksville, MO",msa,MO,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.733430,981.701456,951.458333
469,753923,849,"The Dalles, OR",msa,OR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1757.739178,1788.943167,1885.904065,1838.888889
470,394584,863,"Fallon, NV",msa,NV,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1295.000000
471,394996,915,"Portales, NM",msa,NM,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1122.195307,1101.628147,1057.654564,1071.572024,1046.671512,1099.305556


In [8]:
df.dtypes

RegionID        int64
SizeRank        int64
RegionName     object
RegionType     object
StateName      object
               ...   
7/31/23       float64
8/31/23       float64
9/30/23       float64
10/31/23      float64
11/30/23      float64
Length: 112, dtype: object

In [9]:
df.describe()

,RegionID,SizeRank,1/31/15,2/28/15,3/31/15,4/30/15,5/31/15,6/30/15,7/31/15,8/31/15,...,2/28/23,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23,8/31/23,9/30/23,10/31/23,11/30/23
count,472.000000,472.000000,170.000000,173.000000,177.000000,179.000000,179.000000,178.000000,179.000000,180.000000,...,346.000000,361.000000,372.000000,384.000000,406.000000,411.000000,415.000000,425.000000,445.000000,472.000000
mean,415298.207627,273.686441,1239.825620,1240.669841,1251.146830,1266.885696,1276.013223,1284.256611,1290.350280,1291.137595,...,1903.384062,1898.742328,1908.427805,1906.267933,1886.896644,1892.888767,1897.265795,1891.438391,1877.721153,1851.628367
std,89315.652491,192.924182,413.889910,413.782914,417.808376,430.279161,436.704575,443.645583,446.426265,448.112021,...,1004.878275,984.054607,993.950964,981.101384,943.047137,925.105623,936.407031,974.805502,973.269750,935.988332
min,102001.000000,0.000000,618.854999,621.850858,634.040448,633.276802,623.165150,624.515366,625.812267,645.063429,...,675.528618,753.200396,723.441350,745.512901,748.650235,727.708028,724.647895,726.496330,743.350970,752.666667
25%,394560.500000,118.750000,982.245085,986.598979,993.390743,998.496778,999.149213,999.166349,1002.455958,1006.227359,...,1411.767065,1409.322755,1426.260589,1418.156725,1410.428037,1418.889164,1417.539001,1414.537646,1406.329003,1397.858135
50%,394805.500000,241.500000,1119.640946,1128.291306,1140.587934,1149.357569,1154.063529,1160.902744,1167.827881,1173.401059,...,1725.072590,1707.075023,1707.509644,1701.364516,1700.414955,1706.087275,1716.116918,1708.640942,1696.614785,1675.149130
75%,395063.500000,393.250000,1338.069919,1342.565444,1365.299281,1379.799386,1396.227619,1395.909341,1407.885537,1406.385945,...,2153.244218,2139.535628,2167.521817,2157.862768,2130.928236,2153.958642,2174.036097,2151.869189,2148.939168,2118.656793
max,845167.000000,915.000000,3079.176287,3096.936684,3120.952116,3176.462957,3249.296472,3318.678095,3347.010915,3354.373564,...,15718.660650,15404.494040,15781.904020,15583.894210,14849.608200,14344.093470,14754.112810,16015.296260,16415.672810,15918.888890


In [10]:
min_value = df['11/30/23'].min()
max_value = df['11/30/23'].max()
mean_value = df['11/30/23'].mean()
med_value = df['11/30/23'].median()
std_value = df['11/30/23'].std()
count_value = df['11/30/23'].count()

In [11]:
#| echo: false
print("min:", min_value)
print("max:", max_value)
print("mean:", mean_value)
print("median:", med_value)
print("st. dev:", std_value)
print("N:", count_value)

min: 752.6666667
max: 15918.88889
mean: 1851.6283666211866
median: 1675.1491305
st. dev: 935.9883320322535
N: 472


In [12]:
pd.unique(df['StateName'])

array([nan, 'NY', 'CA', 'IL', 'TX', 'VA', 'PA', 'FL', 'GA', 'MA', 'AZ',
       'MI', 'WA', 'MN', 'CO', 'MD', 'MO', 'NC', 'OR', 'OH', 'NV', 'IN',
       'TN', 'RI', 'WI', 'OK', 'KY', 'LA', 'UT', 'CT', 'AL', 'HI', 'NE',
       'SC', 'NM', 'ID', 'AR', 'IA', 'KS', 'MS', 'ME', 'NH', 'DE', 'AK',
       'NJ', 'SD', 'WV', 'ND', 'VT', 'MT', 'WY'], dtype=object)

In [13]:
df2 = df.melt(id_vars = df.columns[0:5], var_name = "date", value_name = "avg_price")
df2.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,date,avg_price
0,102001,0,United States,country,NaN,1/31/15,1266.059583
1,394913,1,"New York, NY",msa,NY,1/31/15,2233.133615
2,753899,2,"Los Angeles, CA",msa,CA,1/31/15,2571.296547
3,394463,3,"Chicago, IL",msa,IL,1/31/15,1504.096116
4,394514,4,"Dallas, TX",msa,TX,1/31/15,1363.557414


In [14]:
df2['date'] = pd.to_datetime(df2['date'])
df2.head()

/var/folders/r0/k4l4y5td4vv9z4hb8qx7k45m0000gp/T/ipykernel_74647/747068851.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['date'] = pd.to_datetime(df2['date'])


,RegionID,SizeRank,RegionName,RegionType,StateName,date,avg_price
0,102001,0,United States,country,NaN,2015-01-31,1266.059583
1,394913,1,"New York, NY",msa,NY,2015-01-31,2233.133615
2,753899,2,"Los Angeles, CA",msa,CA,2015-01-31,2571.296547
3,394463,3,"Chicago, IL",msa,IL,2015-01-31,1504.096116
4,394514,4,"Dallas, TX",msa,TX,2015-01-31,1363.557414


In [15]:
grouped_df = df2.groupby('StateName')

In [16]:
# Summary statistics for all numeric columns by sex
grouped_df.describe()

RegionID                                                           \
             count           mean       min        25%       50%        75%   
StateName                                                                     
AK           214.0  394453.500000  394327.0  394327.00  394453.5  394580.00   
AL          1605.0  448643.533333  394333.0  394519.00  394598.0  395145.00   
AR           749.0  394790.428571  394590.0  394609.00  394728.0  395042.00   
AZ           856.0  451262.875000  394595.0  394923.25  395096.5  395187.00   
CA          3424.0  428576.656250  394357.0  394841.75  395047.5  395142.00   
CO          1284.0  454493.666667  394405.0  394518.50  394620.5  394908.75   
CT           535.0  394815.600000  394415.0  394669.00  394908.0  394924.00   
DE           214.0  394795.000000  394539.0  394539.00  394795.0  395051.00   
FL          2568.0  424798.708333  394440.0  394759.00  394950.0  395079.25   
GA          2033.0  413673.210526  394306.0  394423.00  394813.0  395126.00   
HI           321.0  514445.000000  394680.0  394680.00  394731.0  753924.00   
IA           749.0  394623.571429  394325.0  394447.00  394542.0  394707.00   
ID           749.0  446097.000000  394399.0  394480.00  394988.0  395173.00   
IL          1177.0  468306.000000  394454.0  394516.00  394734.0  395114.00   
IN          1391.0  422359.384615  394393.0  394565.00  394705.0  395021.00   
KS           749.0  394959.285714  394701.0  394778.00  394981.0  395161.00   
KY           749.0  394736.285714  394406.0  394563.00  394792.0  394949.00   
LA          1177.0  394735.090909  394314.0  394608.00  394761.0  394910.00   
MA           428.0  394779.500000  394361.0  394393.25  394759.5  395145.75   
MD           428.0  484350.750000  394358.0  394474.25  394586.5  484463.00   
ME           214.0  394678.000000  394359.0  394359.00  394678.0  394997.00   
MI          1712.0  417161.437500  394302.0  394580.00  394751.0  394901.25   
MN           642.0  394836.000000  394543.0  394637.00  394844.0  395030.00   
MO          1498.0  394825.571429  394411.0  394604.00  394743.0  395116.00   
MS           642.0  394880.833333  394658.0  394671.00  394831.0  395124.00   
MT           642.0  394577.833333  394386.0  394407.00  394537.0  394733.00   
NC          2354.0  411102.045455  394338.0  394562.00  394697.5  395063.00   
ND           321.0  394892.666667  394585.0  394585.00  394866.0  395227.00   
NE           428.0  394817.750000  394617.0  394751.25  394858.0  394924.50   
NH           214.0  619992.000000  394820.0  394820.00  619992.0  845164.00   
NJ           214.0  394756.000000  394348.0  394348.00  394756.0  395164.00   
NM           963.0  394666.777778  394305.0  394443.00  394588.0  394996.00   
NV           642.0  394673.666667  394444.0  394584.00  394610.0  394775.00   
NY          1070.0  439881.200000  394308.0  394693.00  394972.0  395179.00   
OH          1284.0  432299.833333  394304.0  394472.75  394760.5  395175.25   
OK          1177.0  394820.000000  394300.0  394548.00  394935.0  395133.00   
OR          1498.0  471653.571429  394307.0  394505.00  394800.0  395048.00   
PA          2033.0  394875.421053  394318.0  394666.00  394974.0  395106.00   
RI           107.0  395005.000000  395005.0  395005.00  395005.0  395005.00   
SC          1070.0  394772.600000  394457.0  394597.00  394667.0  395085.00   
SD           321.0  394845.000000  394419.0  394419.00  395013.0  395103.00   
TN          1177.0  394705.181818  394460.0  394474.00  394726.0  394849.00   
TX          3317.0  409264.806452  394299.0  394502.00  394746.0  395054.00   
UT           856.0  394902.000000  394446.0  394768.50  394968.5  395069.50   
VA          1070.0  394914.300000  394392.0  394668.00  395025.0  395194.00   
VT           107.0  394429.000000  394429.0  394429.00  394429.0  394429.00   
WA          1605.0  394891.866667  394378.0  394741.00  394925.0  395113.00   
WI          1177.0  394791.727273  394334.0  394646.00  3948

In [17]:
# Provide the mean for each numeric column by sex
grouped_df.mean(numeric_only = True)

,RegionID,SizeRank,avg_price
StateName,,,
AK,394453.500000,273.000000,1845.229913
AL,448643.533333,296.066667,1196.697249
AR,394790.428571,285.142857,1095.547769
AZ,451262.875000,214.375000,1557.286231
CA,428576.656250,191.437500,2472.280505
CO,454493.666667,348.833333,2346.684403
CT,394815.600000,119.600000,2127.777496
DE,394795.000000,189.000000,1571.385754
FL,424798.708333,174.916667,1668.835798
